In [1]:
## Import all the necessary Libraries required to run this program
import warnings
warnings.filterwarnings("ignore")

import requests
from bs4 import BeautifulSoup
import re

from nltk import tokenize

from fuzzywuzzy import fuzz

from tqdm import tqdm_notebook as tqdm

import operator

import spacy


In [2]:
## Let us remove all the HTML Tags to obtain a much Clearer Text

def striphtml(data):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', str(data))
    cleanr = re.compile(', ')
    cleantext = re.sub(cleanr, ',', str(cleantext))
    cleanr = re.compile('\'')
    cleantext = re.sub(cleanr, '', str(cleantext))
    cleanr = re.compile('\n,')
    cleantext = re.sub(cleanr, ' ', str(cleantext))
    
    return cleantext

def find_answer (sentences, question):
    
    ## By searching through the article we can find where the context related to the Question is present
    scores = []
    index = []

    for i in tqdm(range(len(sentences))) :
        score = fuzz.partial_ratio(question, sentences[i])
        scores.append(score)
        index.append(i)

    dictionary_scores = dict(zip(index, scores))

    probable_answer_index = max(dictionary_scores.items(), key=operator.itemgetter(1))[0]
    score = dictionary_scores[probable_answer_index]
    
    context = []
    
    for i in range(probable_answer_index-2, probable_answer_index+2):
        context.append(sentences[i])
    
    imp_word = []
    
    nlp = spacy.load("en")
    doc = nlp(question)
    
    for token in doc:
        if token.dep_ == 'ROOT':
            imp_word.append(token.text)
        elif token.dep_ == 'obj' or (token.dep_ == 'dobj') or (token.dep_ == 'iobj') or (token.dep_ == 'pobj'):
            imp_word.append(token.text)
        elif (token.dep_ == 'subj') or (token.dep_ == 'nsubj') or (token.dep_ == 'nsubjpass') or (token.dep_ == 'csubj') or (token.dep_ == 'csubjpass') :
            imp_word.append(token.text)
    
    key_phrase = ' '.join(imp_word)
    
    
    for sentence in context :
        score_2 = fuzz.token_set_ratio(sentence, key_phrase)
        
        if score_2 >= 50:
            print(sentence)
            print("Score : {}".format(score))

In [3]:
## The link for the Wikipedia page I webscraped - https://en.wikipedia.org/wiki/R2-D2 
## page_url = "https://en.wikipedia.org/wiki/R2-D2"

page_url = input("Enter from where you need to Extract the Article! : ")

page = requests.get(page_url)

## Let us extract all the Text Data - to understand better Refer README file
soup = BeautifulSoup(page.text)
text_file = soup.find_all('p')

data = striphtml(text_file)

sentences = tokenize.sent_tokenize(data)

Enter from where you need to Extract the Article! : https://en.wikipedia.org/wiki/R2-D2


In [4]:
## Lets search for Questions based on Article for Answers
question = input("Enter any question related to the article: ")

Enter any question related to the article: When was R2-D2 inducted into the Robot Hall of Fame?


In [5]:
find_answer(sentences, question)


[34] R2-D2 was inducted into the Robot Hall of Fame in 2003.
Score : 83
